In [2]:
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import time
#time

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

In [3]:
def import_table():
    """
    return table"选中的表格
    """
    try:
        root = Tk()
        root.filename =  filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.withdraw()
            return table
    except Exception as e:
        root.withdraw()
        print("导入错误")

def save_model(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该词频表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该词频表的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
        else:
            pass
    except Exception as e:
        pass

### 第一步：API导入文书
-----------------------------------------------------------------------
* 此步借助openlaw API的平台来得到文书：http://develop.openlaw.cn/index.jsp


* 个人账号文书的每日上限是1000份文书，因此请控制下列代码运行时的条件。

In [8]:
### API查询模块，最终得到的是一个含有案件号的excel文件
def extract(html_list):
    list_cases = list(map(lambda x:x.text,html_list))
    list_cases_2 = list(map(lambda x:eval(x)["judgements"],list_cases))
    cases_3 = list(np.array(list_cases_2).flatten())
    list_case = []
    for index in range(0,10):
        for item in cases_3[index]:
            elem = item["judgement"]
            list_case.append(elem)
    return list_case

class openlaw():
    def __init__(self):
        self.AppKey = "65d54678b0ec414baf31386fa799cbca"
        self.AppSecret = "90a0af2cb7864ab28e160a89a9398436"
        self.Nonce = str("12sddrobc343fghasf5htl56")
        self.Curtime = str(int(time.time() * 1000))
    
    def openlaw_api_headers(self):
        self.Combined = self.AppSecret + self.Nonce + self.Curtime
        sha1 = hashlib.sha1(self.Combined.encode("utf-8"))
        self.CheckSum = sha1.hexdigest()
        self.headers = {"AppKey":self.AppKey,
                   "Nonce":self.Nonce,
                   "CurTime":self.Curtime,
                   "CheckSum":self.CheckSum}
        return self.headers

    def openlaw_api_search(self):
        target = "http://develop.openlaw.cn/judgement/search"
        keyword = input("请输入需要查询的案例关键词：")
        url = target + "?keyword=" + keyword
        ### 以下为选填参数
        """
        请求参数说明
        参数	参数说明
        keyword	搜索关键字
        court	法院名称
        judgeDateYear	判决年份
        """
        switch = True
        while switch == True:
            court,year = None,None
            paramater_none = input("是否需要其他设定参数（输入Y/N）")
            if paramater_none == "Y":
                pass
            else:
                break
            paramater = input("请输入还需要限定的参数")
            if paramater == "法院名称":
                court = input("哪个法院？")
                url = url + "&court=" + str(court)
            if paramater == "判决年份":
                year = input("哪一年?")
                url = url + "&judgeDateYear=" + str(year)
            paramater_none = input("还需要设定参数吗？（输入Y/N）")
            if paramater_none == "N":
                switch = False
            else:
                switch = True
        headers = self.openlaw_api_headers()
        for index in range(0,11):
            url = url + "&judgeDateBegin=2011-03-01" # 需要修改的地方
            url = url + "&judgeDateEnd=2019-03-01" # 需要修改的地方
            url = url + "&page=10"
        html = requests.get(url,headers = headers)
        return html
    
    def openlaw_api_search_v2(self,keyword):
        #### 设定法院进行爬取
        target = "http://develop.openlaw.cn/judgement/search"
        list_urls = []
        url = target + "?keyword=" + keyword
        with open("courts.txt","r",encoding = "utf-8") as f:
            court_list = eval(f.read())
        #### 选择你需要的法院
        for court in court_list[0:10]:
            url_single = url + "&court=" + str(court)
            list_urls.append(url_single)
        headers = self.openlaw_api_headers()
        html_list = list(map(lambda x:requests.get(x,headers = headers),list_urls))
        list_case = extract(html_list)
        return list_case
        
    def openlaw_api_court(self):
        target = "http://develop.openlaw.cn/judgement/court"
        dicts = {}
        keycourt = input("请输入需要查询的法院：")
        headers = self.openlaw_api_headers()
        dicts["court"] = keycourt
        html = requests.get(target,headers = headers,data = data)
        return html
        
    def openlaw_api_analysis(self):
        target = "http://develop.openlaw.cn/analytic/judgement"
        dicts = {}
        keycourt = input("请输入需要查询的关键字：")
        dicts["keyword"] = keyword
        html = requests.get(target,headers = headers,data = data)
        return html
    
    def openlaw_api_case_detail(self):
        target = "http://develop.openlaw.cn/judgement/detail"
        dicts = {}
        keycourt = input("")
    
o1 = openlaw()
html = o1.openlaw_api_search()

请输入需要查询的案例关键词：杀人
是否需要其他设定参数（输入Y/N）N


In [9]:
print(html.text)

{"judgements":[{"judgement":{"sequence":67548076,"judgeDate":"2017-03-31","litigationType":"Criminal","docType":"Other","causes":["故意杀人"],"id":"f82a690b48e34faea4c8f58846ffe87b","judgeResult":"Unknow","title":"陆志勇故意杀人罪","court":"广东省广州市中级人民法院","judgeWay":"Unknow","caseNo":"（2017）粤01刑初14号","courtId":"b78f91adb9b146a3971c03570fcb04b5"}},{"judgement":{"sequence":50179483,"judgeDate":"2017-05-23","litigationType":"Criminal","docType":"Other","id":"6edac31f47da43d0bc56c513508a9ca9","judgeResult":"Unknow","title":"ＸＸ惠故意杀人一案","court":"黑龙江省高级人民法院","judgeWay":"Unknow","caseNo":"（2017）黑刑核39463447"}},{"judgement":{"sequence":39624552,"judgeDate":"2017-05-31","litigationType":"Criminal","docType":"Other","id":"994dceebe9794bc0ab4e4f39246b2a9b","judgeResult":"Unknow","title":"李立成故意杀人一案","court":"黑龙江省高级人民法院","judgeWay":"Unknow","caseNo":"（2017）黑刑终147号"}},{"judgement":{"sequence":39624571,"judgeDate":"2017-06-13","litigationType":"Criminal","docType":"Other","id":"3bf78154a68241a08851f2cd029c05ba","ju

In [7]:
pd.DataFrame(list(map(lambda x:x["judgement"],json.loads(html.text)["judgements"])))

,caseNo,causes,court,courtId,docType,id,judgeDate,judgeResult,judgeWay,litigationType,sequence,title
0,(2011)粤高法刑一终字第159号,NaN,广东省高级人民法院,274179bf9f3c4d97b6ad254495893d21,Ruling,b12c81d46a3a40e093b9de8532271d95,2011-12-14,Unknow,Unknow,NaN,27750818,27585、林伟权等故意杀人、故意杀人案
1,（2018）辽07刑初33号,[故意杀人],辽宁省锦州市中级人民法院,38665d8aac884d58a41efca864a50992,Other,5723148229a947219f05a8483d785ef0,2018-09-28,Unknow,Unknow,Criminal,65472154,故意杀人
2,（2018）辽07刑初44号号,[故意杀人],辽宁省锦州市中级人民法院,38665d8aac884d58a41efca864a50992,Other,f24798df275a4e5a9c266a384418ebfd,2018-09-29,Unknow,Unknow,Criminal,65472150,故意杀人
3,（2017）冀刑申69号,NaN,河北省高级人民法院,3e740fa43d5d4a71994dca482c370ac5,Notification,4bea08e769c5433abb17d1ed53ec437a,2017-04-19,Unknow,Unknow,Criminal,26608367,袁颖故意杀人
4,（2013）松刑初字第1233号,[故意杀人罪],上海市松江区人民法院,d489e4621ea44365ba7d75aef3361f9d,Verdict,417f1716b5df4b77a8e2b39001f8fdd8,2013-10-10,Unknow,Unknow,Criminal,23406609,李某故意杀人案
5,（2013）沪高刑终字第19号,[故意杀人罪],上海市高级人民法院,ce3870d3927a4689b41ae9cf4d508670,Ruling,bb5914f7a6e246b1899ea9430517131f,2013-03-22,Unknow,Unknow,Criminal,24301236,吴延冲故意杀人案
6,（2012）内刑一复字第13号,[故意杀人罪],内蒙古自治区高级人民法院,814528f1278e4a7b9fc81c620094454a,Ruling,786ecdadaaf44dee9d2e4e362634dda5,2012-04-06,Unknow,Unknow,Criminal,30685926,徐金虎犯故意杀人一案
7,（2012）浦刑初字第2612号,[故意杀人],上海市浦东新区人民法院,05bf8b55bce74272afa19372c5a2b71b,Verdict,c0ce12747a99493fb5b6e4c1cba146dc,2012-07-23,Unknow,Unknow,Criminal,24341357,周利明故意杀人案
8,（2012）台刑初字第046号,[故意杀人罪],台前县人民法院,64ffb738b16b439d957c17b3b9043f3f,Verdict,c16f437fccc24a4ca5263b3079a55461,2012-05-30,Unknow,Unknow,Criminal,25198731,范传臣故意杀人一案
9,（2013）浙刑三终字第28号,[故意杀人罪],浙江省高级人民法院,953dd52647334969a19bc9805c4fece2,Ruling,c395b978ecba4f4098e4b13bf68bbadc,2013-04-02,Unknow,Unknow,Criminal,16814471,吴祖迪故意杀人一案
